# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import numpy as np
import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.build_survey_data.calibration_aliss import get_adjusted_input_data_frame


SyntaxError: invalid syntax (aliss.py, line 382)

In [ ]:
year = 2014
data_year = 2011
reform_key = 'aliss_sante'
scenario_kwargs = dict(year = year, data_year = 2011, reform_key = reform_key)
survey_scenario = SurveyScenario.create(**scenario_kwargs)
# 

In [ ]:
adjusted_scenario_kwargs = dict(scenario_kwargs)
adjusted_scenario_kwargs.update(dict(
    data_year = None,
    input_data_frame = get_adjusted_input_data_frame(reform_key = reform_key[6:])
    ))
adjusted_survey_scenario = SurveyScenario.create(**adjusted_scenario_kwargs) 

In [ ]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]
depenses_ht_tvas = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'tva_taux_')
    ]
tvas = [
    key for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'tva_taux_')
    ]


In [ ]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    ] + depenses_ht_tvas + tvas

In [ ]:
pivot_table = pd.DataFrame()
reference_pivot_table = pd.DataFrame()
adjusted_pivot_table = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
adjusted_aggregates = dict()

for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    reference_pivot_table = pd.concat([
        reference_pivot_table.fillna(0),
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    adjusted_pivot_table = pd.concat([
        adjusted_pivot_table,
        adjusted_survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = (
        survey_scenario.compute_aggregate(variable, reference = True) / 1e9
        if np.isfinite(survey_scenario.compute_aggregate(variable, reference = True)) else 0
        )
    adjusted_aggregates[variable] = adjusted_survey_scenario.compute_aggregate(variable) / 1e9
    


In [ ]:
reference_aggregates['depenses_ht_tva_taux_01_1_1_1_1']

In [ ]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [ ]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [ ]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
         print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

In [ ]:
for key in sorted(aggregates.keys()):
    if round(adjusted_aggregates[key] - aggregates[key], 3) != 0:
         print key, ': ', round(adjusted_aggregates[key], 3), ' - ', round(aggregates[key], 3), ' = ', round(adjusted_aggregates[key] - aggregates[key], 3)

In [ ]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [ ]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

In [ ]:
pivot_table - reference_pivot_table

In [ ]:
adjusted_pivot_table - pivot_table